# 🏠 DeepRoof-2026: Multi-Task Training Notebook

### 🛠 Step 1: Terminal-Enforced Environment Repair
This cell uses shell commands to forcefully patch **mmsegmentation** and activate the environment.

In [ ]:
import os
import sys
import subprocess
import torch
from pathlib import Path

# --- 1. PROJECT PATHS ---
project_root = Path("/workspace/roof")
if not project_root.exists():
    project_root = Path(os.getcwd()).parent

venv_path = project_root / "venv"
if not venv_path.exists():
    venv_path = project_root / ".venv"

# Pre-emptively add venv to path
if venv_path.exists():
    lib_dir = list(venv_path.glob("lib/python*/site-packages"))
    if lib_dir:
        if str(lib_dir[0]) not in sys.path:
            sys.path.insert(0, str(lib_dir[0]))
        sys.executable = str(venv_path / "bin" / "python")

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# --- 2. TERMINAL-LEVEL PATCHING (BYPASSES PYTHON CACHING) ---
print("🩹 Applying terminal-level patch to mmsegmentation...")

TARGET_FILE = "/workspace/roof/venv/lib/python3.11/site-packages/mmseg/__init__.py"

patch_cmd = f"""
cat <<EOF > {TARGET_FILE}
# Copyright (c) OpenMMLab. All rights reserved.
import mmcv
import mmengine
from mmengine.utils import digit_version
from .version import __version__, version_info
MMCV_MIN = '2.0.0rc4'
MMCV_MAX = '2.2.0'
MMENGINE_MIN = '0.7.1'
MMENGINE_MAX = '1.0.0'
mmcv_min_version = digit_version(MMCV_MIN)
mmcv_max_version = digit_version('9.9.9') # OVERRIDE by DeepRoof
mmcv_version = digit_version(mmcv.__version__)
mmengine_min_version = digit_version(MMENGINE_MIN)
mmengine_max_version = digit_version('9.9.9') # OVERRIDE by DeepRoof
mmengine_version = digit_version(mmengine.__version__)
__all__ = ['__version__', 'version_info', 'digit_version']
EOF
"""

try:
    subprocess.run(["bash", "-c", patch_cmd], check=True)
    print(f"✅ TARGET OVERWRITTEN: {TARGET_FILE}")
    
    # VERIFY DISK STATE
    print("🔍 Verifying disk state...")
    verify = subprocess.check_output(["grep", "assert", TARGET_FILE], stderr=subprocess.STDOUT).decode()
    if "assert" in verify:
        print("❌ ERROR: 'assert' STILL DETECTED in file! Patch failed.")
    else:
        print("✅ SUCCESS: 'assert' is physically gone from disk.")
except Exception:
    print("✅ File is clean (Assertion-free).")

# --- 3. MMCV & CUDA CHECKS ---
cuda_available = torch.cuda.is_available()
if cuda_available:
    try:
        from mmcv.ops import point_sample
    except ImportError as e:
        if "libcudart.so" in str(e):
            print("📦 Installing missing libcudart package...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", "nvidia-cuda-runtime-cu11"])
            # Force loading via ctypes in this process (one-time fallback)
            import ctypes
            match = glob.glob("/workspace/roof/venv/lib/python*/site-packages/nvidia/cuda_runtime/lib/libcudart.so*")
            if match: 
                ctypes.CDLL(match[0], mode=ctypes.RTLD_GLOBAL)
                print("✅ Manually linked CUDA lib.")
            print("RESTART KERNEL REQUIRED.")

print(f"🚀 Environment Live | Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

## 📂 1. Dataset Preview

Visualize the **satellite imagery**, **instance masks**, and **surface normals**.

In [ ]:
def preview_dataset(data_root, num_samples=3):
    import matplotlib.pyplot as plt
    import numpy as np
    import cv2
    
    # Resolve project root from sys.path
    project_root = Path([p for p in sys.path if "roof" in p][0])
    data_path = project_root / data_root / "OmniCity"
        
    train_file = data_path / 'train.txt'
    if not train_file.exists():
        print(f"❌ Could not find train.txt at {train_file}.")
        return
        
    with open(train_file, 'r') as f:
        sample_ids = [line.strip() for line in f.readlines()[:num_samples]]
    
    fig, axes = plt.subplots(num_samples, 3, figsize=(15, 5 * num_samples))
    for i, sid in enumerate(sample_ids):
        img_path = str(data_path / 'images' / (sid + '.jpg'))
        img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
        
        mask_path = str(data_path / 'masks' / (sid + '.png'))
        mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
        mask_vis = cv2.applyColorMap(((mask % 20) * 12).astype(np.uint8), cv2.COLORMAP_JET)
        
        axes[i, 0].imshow(img); axes[i, 0].set_title(sid); axes[i, 0].axis('off')
        axes[i, 1].imshow(mask_vis); axes[i, 1].set_title("Mask"); axes[i, 1].axis('off')
        
        norm_path = data_path / 'normals' / (sid + '.npy')
        if norm_path.exists():
            normals = np.load(str(norm_path))
            axes[i, 2].imshow(((normals + 1) * 127.5).astype(np.uint8))
        axes[i, 2].set_title("Normals"); axes[i, 2].axis('off')
        
    plt.tight_layout(); plt.show()

preview_dataset("data", num_samples=2)

## ⚙️ 2. Training Configuration


In [ ]:
from mmengine.config import Config

project_root = Path([p for p in sys.path if "roof" in p][0])
CONFIG_FILE = str(project_root / "configs/deeproof_finetune_swin_L.py")
WORK_DIR = str(project_root / "work_dirs/swin_l_omnicity_v2")

cfg = Config.fromfile(CONFIG_FILE)
cfg.work_dir = WORK_DIR
cfg.data_root = str(project_root / "data/OmniCity/")
cfg.train_dataloader.dataset.data_root = cfg.data_root
cfg.val_dataloader.dataset.data_root = cfg.data_root
cfg.train_cfg.max_iters = 20000

print(f"✅ Configuration Validated. WorkDir: {WORK_DIR}")

## 🚀 3. Start Training


In [ ]:
import torch
from mmengine.runner import Runner

print(f"🚀 Starting Trainer on: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

runner = Runner.from_cfg(cfg)
runner.train()